In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train_data = train_data.drop(['PassengerId', 'Cabin', 'Ticket', 'Embarked', 'Name'], axis = 1)
train_data.Age = train_data.Age.fillna(train_data.Age.median())
train_data.Sex = train_data.apply(lambda sex: sex == 'male')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    bool   
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: bool(1), float64(2), int64(4)
memory usage: 42.8 KB


In [6]:
test_data = test_data.drop(['PassengerId', 'Cabin', 'Ticket', 'Embarked', 'Name'], axis = 1)
test_data.Age = test_data.Age.fillna(train_data.Age.median())
test_data.Fare = test_data.Fare.fillna(train_data.Fare.median())
test_data.Sex = test_data.apply(lambda sex: sex == 'male')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    bool   
 2   Age     418 non-null    float64
 3   SibSp   418 non-null    int64  
 4   Parch   418 non-null    int64  
 5   Fare    418 non-null    float64
dtypes: bool(1), float64(2), int64(3)
memory usage: 16.9 KB


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import numpy as np

In [8]:
X = train_data.drop(['Survived'], axis = 1)
X = torch.from_numpy(X.astype(np.float32).to_numpy())
X.requires_grad = True

y = train_data.Survived
y = torch.from_numpy(y.astype('int').to_numpy())
y = y.long()

In [9]:
model = nn.Sequential(
    nn.Linear(6, 10),
    nn.BatchNorm1d(10),
    nn.ReLU(),
    nn.Linear(10, 20),
    nn.BatchNorm1d(20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.BatchNorm1d(10),
    nn.ReLU(),
    nn.Linear(10, 2),
    nn.Sigmoid())

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

X = X.to(device)
y = y.to(device)
model = model.to(device)

In [10]:
n_epochs = 20000

for epoch in range(n_epochs):
    
    output = model(X)
    loss = loss_fn(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0: print(loss)


tensor(0.6825, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4861, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4618, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4550, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4552, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4505, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4483, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4477, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4534, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4471, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4458, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4470, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4485, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4457, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4442, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4483, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4428, device='cuda:0', grad_fn=<NllLossBackward

In [11]:
# https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
def confusion(prediction, truth):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

true_pos, false_pos, true_neg, false_neg = confusion(output.max(dim = 1).indices, y)

In [13]:
print("Accuracy:", (true_pos + true_neg) / (true_pos + true_neg + false_neg + false_pos)) 

Accuracy: 0.8664421997755332
